###  free variables (aka nonlocal variables)

In [2]:
from typing import Callable  # type hinting


def inspect_vars(o: Callable) -> None:
    for i in ("co_varnames", "co_freevars"):        #???
        print(i, eval("o.__code__." + i))

In [6]:
Average = float  # type aliasing


def maker() -> Callable[[float], float]  :   #returna una funzione(ie: avg)
    so_far = []  #so_far cant be a globalvar

    def avg(new_value: float) -> float:
        so_far.append(new_value)
        return sum(so_far) / len(so_far)

    return avg


av = maker( )
print(av(9))
print(av(11))
print(av(40))

9.0
10.0
20.0


In [ ]:
inspect_vars(av)

co_varnames ('new_value',)
co_freevars ('so_far',)


In [7]:
def maker_buggy() -> Callable[[float], Average]:
    _sum = 0
    _count = 0

    def avg(new_value: float) -> Average:
        _sum += new_value
        _count += 1                 #stesso errore di prima con b e foo
        return _sum / _count

    return avg


av = maker_buggy()
av(9)

#prima in maker non dava errore e manteneva il risultato perchè
#con list se assegno =[] non cambio il ID, cambio gli elementi tipo.
#mentre con i numeri, che sono immutable, così cambio ID

UnboundLocalError: local variable '_sum' referenced before assignment

In [ ]:
inspect_vars(av)

co_varnames ('new_value', '_sum', '_count')
co_freevars ()


In [7]:
def maker_right() -> Callable[[float], Average]:
    _sum = 0
    _count = 0

    def avg(new_value: float) -> Average:
        nonlocal _sum, _count       # <----
        _sum += new_value
        _count += 1
        return _sum / _count

    return avg                               


av = maker_right()
print(av(9))
print(av(11))
print(av(40))


9.0
10.0
20.0


In [ ]:
inspect_vars(av)

### closure
è una funzione che usa delle freevariables. cioè variabili che possono restare anche fuori dal defining scope (che in questo caso è lo scope di maker!).

A closure is a function that retains the bindings of the free variables that exist when the function is defined, so that they can be used later when the function is invoked and the **defining scope** is no longer available.